### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:

- оценка 2 и меньше - низкий рейтинг

- оценка 4 и меньше - средний рейтинг

- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

ratings = pd.read_csv(r'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [2]:
classify = lambda x: 'низкий рейтинг' if x <= 2 else 'высокий рейтинг' if x >= 4.5 else 'средний рейтинг'
ratings['class'] = ratings['rating'].apply(classify)
ratings.head(15)

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг
5,1,1263,2.0,1260759151,низкий рейтинг
6,1,1287,2.0,1260759187,низкий рейтинг
7,1,1293,2.0,1260759148,низкий рейтинг
8,1,1339,3.5,1260759125,средний рейтинг
9,1,1343,2.0,1260759131,низкий рейтинг


### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [3]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

def geo_classifiyer(row, geo_data = geo_data, column = 'keyword'):
    """
    Гео-классификатор принимает на вход словарь с опредением регионов и наименование столбца,
    по которому необходимо провести классификацию. Возвращает либо регион, либо значение udefined.
    """
    for key, value in geo_data.items():
        for item in value:
            if item in row[column]:
                return (key)
    return ('udefined')

keywords = pd.read_csv(r'keywords.csv')
keywords.tail()

,keyword,shows
99995,эльдорадо старый оскол,3705
99996,frigate для firefox,3630
99997,укрсиб,3630
99998,погода в ялте на неделю,3688
99999,ггму,3630


In [4]:
keywords['region'] = keywords.apply(geo_classifiyer, geo_data = geo_data, column = 'keyword', axis=1)
keywords.tail()

,keyword,shows,region
99995,эльдорадо старый оскол,3705,udefined
99996,frigate для firefox,3630,udefined
99997,укрсиб,3630,udefined
99998,погода в ялте на неделю,3688,udefined
99999,ггму,3630,udefined


In [5]:
#Проверочная выборка для поискового запроса со словом хабаровск
keywords[keywords['keyword'].str.contains('хабаровск')].head()

,keyword,shows,region
2175,дром хабаровск,98361,Дальний Восток
3687,авито хабаровск,63736,Дальний Восток
6260,фарпост хабаровск,40962,Дальний Восток
8039,погода в хабаровске,33437,Дальний Восток
9084,хабаровск,29655,Дальний Восток


In [6]:
#проверочный подсчет значений в столбе region
keywords['region'].value_counts()

udefined          99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

### Задание 3

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы grouplens.org.../movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.
2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    * для каждой строки пройдите по всем годам списка years
    * если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    * если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’
4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [7]:
#Из указанных файлов берем только необходимые столбцы с Id, названием и рейтингом фильмов
ratings = pd.read_csv(r'ratings.csv', usecols=['movieId', 'rating'])
movies = pd.read_csv(r'movies.csv', usecols=['movieId', 'title'])
print(ratings.head(), '\n', movies.head())

   movieId  rating
0       31     2.5
1     1029     3.0
2     1061     3.0
3     1129     2.0
4     1172     4.0 
    movieId                               title
0        1                    Toy Story (1995)
1        2                      Jumanji (1995)
2        3             Grumpier Old Men (1995)
3        4            Waiting to Exhale (1995)
4        5  Father of the Bride Part II (1995)


In [8]:
years = [str(x) for x in range(1950, 2011)]
def production_year(title):
    """
    Функция, возвращающая год производства, если он попадает в промежуток между 1950 и 2010 включительно,
    в противном случает возвращает 1900.
    """
    for item in years:
        if item in title:
            return (item)
    return ('1900')

#добавляем столбец с годом производства
movies['year'] = movies['title'].apply(production_year)
movies.tail()

,movieId,title,year
9120,162672,Mohenjo Daro (2016),1900
9121,163056,Shin Godzilla (2016),1900
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,1900
9123,164977,The Gay Desperado (1936),1900
9124,164979,"Women of '69, Unboxed",1900


In [9]:
#По ключу movieId объединяем с таблицей рейтингов и, группируя по году производства, считаем средний рейтинг.
#Сортировка по убыванию среднего рейтинга.
ratings.merge(movies, how='left', on='movieId').groupby('year')['rating'].mean().sort_values(ascending = False)

year
1957    4.014241
1972    4.011136
1952    4.000000
1954    3.994220
1951    3.983539
          ...   
2005    3.448434
2003    3.444777
1996    3.426600
1997    3.415764
2000    3.356104
Name: rating, Length: 62, dtype: float64